In [1]:
import os
os.environ["DISPLAY"] = ':1001'

In [34]:
import napari
viewer=napari.Viewer()

In [ ]:
import imageio as io
marker=io.imread('/mnt/efs/aimbl_2025/student_data/S-LS/merged/img_001_merged.tif')
annotation=io.imread('/mnt/efs/aimbl_2025/student_data/S-LS/binary_rings-gt-tiff/img_000_binary_gt_positive.tif')
ring=io.imread('/mnt/efs/aimbl_2025/student_data/S-LS/binary_rings-predicted-tiff/img_000_binary_positive.tif')
# raw=io.imread('/mnt/efs/aimbl_2025/student_data/S-LS/raw_bacteria/img_001_bacteria.tif')


viewer.add_image(raw, name="raw_image")
# viewer.add_image(marker, channel_axis=0, name='marker')
# viewer.add_image(marker, name='marker')
viewer.add_labels(ring, name='prediction')
viewer.add_labels(annotation, name='annotation')

/tmp/ipykernel_763347/3513194516.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  marker=io.imread('/mnt/efs/aimbl_2025/student_data/S-LS/merged/img_001_merged.tif')
/tmp/ipykernel_763347/3513194516.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  annotation=io.imread('/mnt/efs/aimbl_2025/student_data/S-LS/binary_rings-gt-tiff/img_000_binary_gt_positive.tif')
/tmp/ipykernel_763347/3513194516.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `im

<Labels layer 'annotation' at 0x717dce8b7d90>

You can convert the layer dtype in the right-click contextual menu of the layer list.


In [9]:
print(marker.shape)


(2, 2208, 2752)


Here’s a quick, bacteria-focused rundown of the metrics you mentioned—what they mean, how they’re computed, and when to use them. I’ll show tiny numeric examples so the intuition sticks.

Pixel-level vs Instance-level (important!)

Pixel-level: treat masks as binary images (bacteria vs background). Good for “overall foreground area” quality.

Instance-level: treat each bacterium as its own object (needs instance labels). Good for “did we detect each bacterium, and how well?”

You can (and often should) report one of each, because they answer different questions.

Pixel-level metrics (binary masks)
Precision (Positive Predictive Value)

What it asks: Of all pixels I predicted as bacteria, how many were actually bacteria?

Formula: precision = TP / (TP + FP)

Example: You predicted 120k bacteria pixels; 100k are truly bacteria, 20k are background → precision = 100k / (100k+20k) = 0.833.
Use when: You want to penalize “extra” smear/bleed of the mask into background.

Recall (Sensitivity)

What it asks: Of all true bacteria pixels, how many did I capture?

Formula: recall = TP / (TP + FN)

Example: There are 130k true bacteria pixels; you captured 100k → recall = 100k / (100k+30k) = 0.769.
Use when: Missing bacteria area is costly.

F1 score (same as Dice for binary segmentation)

What it asks: Balanced overlap score combining precision and recall.

Formula: F1 = 2·precision·recall / (precision + recall) = 2TP/(2TP+FP+FN)

Example using above: F1 ≈ 2·0.833·0.769 / (0.833+0.769) ≈ 0.800.
Use when: Single headline number for binary mask overlap—this is usually the most informative one-liner for “total bacteria” masks.

Mean IoU (a.k.a. Jaccard)

What it asks: How big is the intersection relative to the union?

Formula: IoU = TP / (TP + FP + FN)

Example with 100k TP, 20k FP, 30k FN: IoU = 100k/(100k+20k+30k)= 0.667.
Use when: You want a stricter overlap score than Dice; common in papers/benchmarks.

⚠️ Plain Accuracy can look great just because most pixels are background. Avoid it as a headline in segmentation.

Instance-level metrics (objects)

These require instance labels (0=background, 1..K for each bacterium). First you match predicted objects to GT objects (usually by IoU ≥ 0.5).

Object Precision / Recall / F1

What it asks: Did we detect the right objects (not just pixels)?

TP: predicted object matched a GT object at IoU ≥ τ (e.g., 0.5)

FP: predicted object with no match

FN: GT object with no matched prediction

Formulas: same as pixel precision/recall/F1 but counting objects instead of pixels.

Example: In one image:

GT bacteria = 50

Predicted = 47

Matches at IoU≥0.5 = 42
→ TP=42, FP=5, FN=8 → precision=42/47=0.894, recall=42/50=0.840, F1≈0.866.
Use when: You care about finding the right number of bacteria.

Counting error

What it asks: How far off is my count?

Per image: |#pred − #gt|

Dataset: mean absolute error (MAE) or signed error (bias)

Example: GT=50, Pred=47 → |Δ|=3.
Use when: You care about counts (e.g., density estimates).

Mean IoU of matched instances

What it asks: For the correctly detected bacteria, how good are the shapes?

Compute IoU per matched pair; take the mean.

Example: 42 matched bacteria have IoUs averaging 0.74.
Use when: Shape/boundary quality matters (separate from detection).

Note: This can stay flat even while detection improves (you saw this earlier).

So…what’s “most accurate” for your comparison?

You said: “comparing my mask with the StarDist mask of total bacteria” → sounds like binary foreground vs background comparison, not per-object analysis.

If that’s right, the best single headline metric is Dice (F1).

It balances missing area (FN) and extra area (FP), is robust to background imbalance, and is the standard for binary segmentation overlap.

Also report IoU (it’s stricter and widely recognized).

If you also care about detection/counts, add instance-level object F1 (at IoU≥0.5) and counting error.

Quick recommendation set

Primary: Dice (F1) and IoU on the binary masks.

Optional (if you have instances): Object F1 @ IoU≥0.5 and counting error.

Optional boundary quality: Boundary F1 (BF-score) if edge sharpness matters.

Mini cheat-sheet

Binary overlap quality: Dice/F1 (↑ better), IoU (↑ better).

Too much foreground? Precision low.

Missing bacteria area? Recall low.

Too many/few bacteria? Look at object recall and counting error.

Shapes on matched objects: Mean IoU of matches (or boundary F1).

If you want, I can adapt your earlier evaluation code to output both binary Dice/IoU and instance-level object F1/counting error in one go, using the same folder structure you already set up.